In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
docs_raw[1]['documents'][0].keys()

dict_keys(['text', 'section', 'question'])

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model='gpt-4.1',
    messages=[{"role": "user", "content": q}]
)
response.choices[0].message.content

'The ability to enroll in a course after it has started depends on the specific course and the policies of the institution or platform offering it. Here are some steps you can take:\n\n1. **Check Official Guidelines:** Look at the course’s website or platform for information about late enrollment or add/drop periods.\n2. **Contact Support or the Instructor:** Reach out to the course administrator, instructor, or admissions office directly. They can provide you with the best guidance on whether late enrollment is possible.\n3. **Consider Implications:** If late enrollment is allowed, ask about any missed material and how you can catch up.\n\nIf you tell me which course or institution you’re referring to, I may be able to provide more specific advice!'

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
def rag(query):
    search_results = search(query)
    #print(search_results)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
query = 'the course has already started, can I still enroll?'
rag(query)

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects and try not to leave everything for the last minute.'

In [16]:
query = 'how do I run Kafka?'
rag(query)

'To run Kafka as a Java producer or consumer in the terminal, you should navigate to your project directory and execute the following command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the specific name of your JAR file.'

In [17]:
query = 'the course has already finished, can I still enroll?'
rag(query)

'Yes, you can still follow the course even after it has finished. All the course materials will remain available, allowing you to learn at your own pace. You can also continue to review the homework and start preparing for future cohorts. Moreover, you can begin working on your final capstone project.'

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
from elasticsearch import Elasticsearch

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [20]:
es_client = Elasticsearch('http://localhost:9200')

In [21]:
es_client.info

<bound method Elasticsearch.info of <Elasticsearch(['http://localhost:9200'])>>

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
} 

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
from tqdm.auto import tqdm

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [26]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [27]:
index_name

'course-questions'

In [28]:
query = 'I just disovered the course. Can I still join it?'

In [29]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
}

In [30]:
response = es_client.search(index=index_name, body=search_query)

In [31]:
response

ObjectApiResponse({'took': 71, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266, 'hits': [{'_index': 'course-questions', '_id': 'JV1sqZYBHFncDFVLSjXR', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'Kl1sqZYBHFncDFVLSjXk', '_score': 54.057133, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next 

In [32]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'JV1sqZYBHFncDFVLSjXR',
  '_score': 72.849266,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'Kl1sqZYBHFncDFVLSjXk',
  '_score': 54.057133,
  '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after 

In [33]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [34]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [35]:
def elastic_search(query, index_name=''):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs
    

In [36]:
query = 'I just disovered the course. Can I still join it?'
res = elastic_search(query)

In [37]:
res

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [38]:
query = 'How to run Kafka?'
res = elastic_search(query)

In [39]:
res

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [40]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
query = 'How to run Kafka?'
res = rag(query)

In [42]:
res

'To run a Java Kafka producer or consumer in the terminal, navigate to the project directory and execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. This command applies to Java Kafka setups specifically.'

In [43]:
query = 'I just disovered the course. Can I still join it?'
res = rag(query)

In [44]:
res

"Yes, you can still join the course after the start date. Even if you haven't registered, you're eligible to submit the homework. However, keep in mind that there will be deadlines for submitting the final projects, so it's important not to wait until the last minute."